In [ ]:
%matplotlib inline
from __future__ import print_function
import gc
import math
import os
import random
import sys
import traceback
import sklearn.metrics
import skimage.color
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import ndimage

import improc
import convnet
import mutate
import convevo
import darwin

In [ ]:
reload (improc)
reload (convnet)
reload (mutate)
reload (convevo)
reload (darwin)

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

# Enumerate Images
Image names are sequential, so add every tenth image to the validation set based on filename.

In [ ]:
training = []
test = []

for root, dirs, files in os.walk('captures'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        # Find all the image files, split into test and training.
        if low_name.endswith(".png"):
            if low_name.endswith("0.png"):
                test.append(path)
            else:
                training.append(path)

print("Training:", len(training), "Test:", len(test))
print(training[:2])
print(test[:2])

# Image Processing
Each image file contains a color image (top half), and an encoded depth image (bottom half)
<img src="testing/IMG_2114.PNG">
* Note: The image may also contain the orientation data. If so it is encoded in the first two pixels of the depth image. If the first pixel of the depth image is red, the second has the x, y, z, w quaternion components encoded in the r,g,b,a values.

The improc module contains functions for splitting the image, decoding the depth back into floating point millimeters, and for filling in gaps.

In [ ]:
COLOR_CHANNELS = 3

def load_image(image_path):
    combined_image = ndimage.imread(image_path).astype(np.float32)
    color_image, depth_image = improc.split(combined_image)
    color_image = color_image[:, :, 0 : COLOR_CHANNELS] / improc.BYTE_MAX # Discard alpha and normalize
    depths, attitude = improc.decode_depth(depth_image)
    return (color_image, depths, attitude)

In [ ]:
example_image, example_depth, example_attitude = load_image("testing/IMG_2114.PNG")
plt.imshow(example_image)
print(example_image.shape, example_image.dtype)

In [ ]:
plt.imshow(example_depth)
print(example_depth.shape, example_depth.dtype)
print(example_attitude)

In [ ]:
#CIELAB image component scales:
L_MAX = 100
AB_SCALE_MAX = 127
def rgb2lab_normalized(image):
    lab_image = skimage.color.rgb2lab(image)
    return (lab_image / [L_MAX / 2, AB_SCALE_MAX, AB_SCALE_MAX]) - [1, 0, 0]

In [ ]:
example_lab = rgb2lab_normalized(example_image)
plt.imshow(example_lab[:,:,0], cmap='Greys_r')

In [ ]:
plt.imshow(example_lab[:,:,1], cmap='Greys_r')

In [ ]:
def compute_average_depth():
    depth_averages = []

    for path in training:
        _, depth, _ = load_image(path)
        depth_averages.append(np.nanmean(depth))
        if len(depth_averages) % 1000 == 0:
            print("Image", len(depth_averages))
    return np.nanmean(depth_averages)

# Precomputed via compute_average_depth()
MEAN_DEPTH = np.float32(1688.97)

print(MEAN_DEPTH)

# Depth labels
Want more precision for nearby things, so use progressively expanding buckets for labels, so if smallest bucket has size s and each succesive bucket is larger by a factor F then:

improc.MAX_DEPTH == sF<sup>0</sup> + sF<sup>1</sup> + sF<sup>2</sup> + ... + sF<sup>label count - 1</sup>

So, plug into sum of geometric series formula:

improc.MAX_DEPTH == s * (1 - F<sup>label count</sup>) / (1 - F)

Since there are two unknowns we can choose either the factor or the bucket size. A factor of 1.3 resulted in buckets that seemed about right.

In [ ]:
def size_for_factor(factor, buckets):
    return improc.MAX_DEPTH * (1 - factor) / (1 - factor ** buckets)

def depth_label_boundaries(factor, buckets):
    boundaries = []
    size_sum = 0
    bucket_size = size_for_factor(factor, buckets)
    for i in range(buckets):
        size_sum += bucket_size
        boundaries.append(size_sum)
        bucket_size *= factor
    return boundaries

DEPTH_LABEL_COUNT = 20
DEPTH_BUCKET_SCALE_FACTOR = 1.3
DEPTH_BOUNDARIES = depth_label_boundaries(DEPTH_BUCKET_SCALE_FACTOR, DEPTH_LABEL_COUNT)
print(DEPTH_BOUNDARIES[:5])

def depth_label_index(depth):
    for i, boundary in enumerate(DEPTH_BOUNDARIES):
        if depth < boundary:
            return i
    return DEPTH_LABEL_COUNT - 1

def depth_label(depth, labels=None):
    if labels is None:
        labels = np.zeros(shape=(DEPTH_LABEL_COUNT), dtype=np.float32)
    labels[depth_label_index(depth)] = 1
    return labels

def depth_label_image(depths):
    labeled = depths.copy()
    for y in xrange(depths.shape[0]):
        for x in xrange(depths.shape[1]):
            labeled[y,x] = depth_label_index(depths[y,x])
    return labeled

print("Mean depth label:", depth_label(MEAN_DEPTH))

In [ ]:
plt.imshow(depth_label_image(example_depth))

In [ ]:
class ImageSampler(object):
    """Wrap an image for sampling."""
    def __init__(self, image_file, sample_height, sample_width, image_cache=None, half_valid_check=2, tolerance=1):
        cached = None
        if image_cache:
            cached = image_cache.get(image_file)

        if cached:
            self.image, self.depth = cached
        else:
            self.image, self.depth, _ = load_image(image_file)
            self.image = rgb2lab_normalized(self.image)
        
        if image_cache and not cached:
            image_cache[image_file] = (self.image, self.depth)

        self.y = 0
        self.x = 0
        self.sample_height = sample_height
        self.sample_width = sample_width
        self.depth_offset_y = (sample_height + 1) / 2
        self.depth_offset_x = (sample_width + 1) / 2
        self.height = self.image.shape[0]        
        self.width = self.image.shape[1]
        self.half_valid_check = half_valid_check
        self.tolerance = tolerance
        
    def depth_value(self, y_offset=0, x_offset=0):
        return self.depth[self.y + self.depth_offset_y + y_offset, self.x + self.depth_offset_x + x_offset]
        
    def sample(self, inputs, labels, index):
        patch = self.image[self.y : self.y + self.sample_height, self.x : self.x + self.sample_width, 0:1]
        inputs[index] = patch
        depth_label(self.depth_value(), labels[index])
        self.advance()
    
    def advance(self):
        self.x += 1
        if self.x + self.sample_width >= self.width:
            self.x = 0
            self.y += 1
    
    def next_sample(self):
        c = self.half_valid_check
        while self.y + self.sample_height < self.height:
            depth_y = self.y + self.depth_offset_y
            depth_x = self.x + self.depth_offset_x
            # Check that the sample is from a clean part of the image.
            sum = np.sum(np.isnan(self.depth[depth_y - c : depth_y + c, depth_x - c: depth_x + c]))
            if sum <= self.tolerance:
                return True
            self.advance()
        return False

In [ ]:
class BatchSampler(object):
    """Created sample batches for a set of image files"""
    def __init__(self, image_files, sample_height, sample_width, samplers_count=100, image_cache=None):
        self.files = image_files
        self.samplers_count = samplers_count
        self.sample_height = sample_height
        self.sample_width = sample_width
        self.image_cache = image_cache
        self.reset()
        
    def sample(self, inputs, labels, index):
        sampler = self.samplers[self.sample_index]
        if sampler and not sampler.next_sample():
            sampler = None

        while sampler is None:
            path = self.files[self.file_index]
            sampler = ImageSampler(path, self.sample_height, self.sample_width, self.image_cache)
            self.file_index = (self.file_index + 1) % len(self.files)
            if not sampler.next_sample():
                sampler = None
                print ("No samples in", path)
            else:
                self.samplers[self.sample_index] = sampler

        self.sample_index = (self.sample_index + 1) % len(self.samplers)
        sampler.sample(inputs, labels, index)
        
    def sample_batch(self, inputs, labels, batch_size):
        labels.fill(0)
        for b in xrange(batch_size):
            self.sample(inputs, labels, b)
            
    def reset(self):
        self.sample_index = 0
        self.file_index = 0
        self.samplers = [None] * self.samplers_count

In [ ]:
example_cache = {}

In [ ]:
SAMPLE_SIZE = 101
batcher = BatchSampler(["testing/IMG_2114.PNG", "testing/IMG_3410.PNG"], SAMPLE_SIZE, SAMPLE_SIZE, 2, example_cache)

In [ ]:
BATCH_SIZE = 100

inputs = np.ones(shape=(BATCH_SIZE, SAMPLE_SIZE, SAMPLE_SIZE, COLOR_CHANNELS), dtype=np.float32)
labels = np.zeros(shape=(BATCH_SIZE, DEPTH_LABEL_COUNT), dtype=np.float32)

for _ in xrange(100):
    batcher.sample_batch(inputs, labels, BATCH_SIZE)
    
del example_cache

In [ ]:
plt.imshow(inputs[1,:,:,0], cmap='Greys_r')
print(inputs[1].shape)
print(labels[1])

# Data Management

In [ ]:
data_files = {
    "image_size": (SAMPLE_SIZE, SAMPLE_SIZE, 1),
    "depth_labels": DEPTH_LABEL_COUNT,
    "train_files": np.array(training),
    "test_files": np.array(test)
}

del training
del test

In [ ]:
def setup_cross_validation(data, train_count, valid_count, test_count=None, label_count=None, seed=None):
    cross_data = data.copy()

    if seed:
        np.random.seed(seed)
    
    if label_count:
        cross_data["depth_labels"] = label_count

    paths = cross_data["train_files"][:]
    permutation = np.random.permutation(paths.shape[0])
    paths = paths[permutation]

    cross_data["train_files"] = paths[:train_count]
    cross_data["valid_files"] = paths[train_count:train_count + valid_count]

    if test_count is not None:
        cross_data["test_files"] = data["test_files"][:test_count]

    return cross_data

# Graph Setup

In [ ]:
def setup_graph(
    batch_size,
    image_shape,
    label_count,
    layer_stack
):
    graph = tf.Graph()
    with graph.as_default():
        input_shape = (batch_size,) + image_shape
        output_shape = (batch_size, label_count)
        train   = tf.placeholder(tf.float32, shape=input_shape)
        labels = tf.placeholder(tf.float32, shape=output_shape)
        verify  = tf.placeholder(tf.float32, shape=input_shape)

        layers = layer_stack.construct(input_shape)
        l2_loss = 0
        
        for layer in layers:
            layer.setup_parameters()
            l2_loss = layer.update_loss(l2_loss)
        
        def model(nodes, train):
            for layer in layers:
                nodes.append(layer.connect(nodes[-1], train))
            return nodes[-1]

        logits = model([train], True)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels)) + l2_loss
        
        global_step = tf.Variable(0)
        optimizer = layer_stack.construct_optimizer(global_step)
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "labels": labels,
            "loss": loss,
            "optimizer": optimizer.minimize(loss, global_step=global_step),

            # Predictions for training and verification (validation or test)
            "predictions": tf.nn.softmax(logits),
            "verify": verify,
            "verify_predictions": model([verify], False)
        }
    return info

# Graph Execution

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
def batch_accuracy(session, graph_info, batcher, inputs, labels, batch_size, batch_count):
    total_accuracy = 0
    for b in xrange(batch_count):
        batcher.sample_batch(inputs, labels, batch_size)
        predictions = session.run([graph_info["verify_predictions"]], feed_dict={graph_info["verify"] : inputs})[0]
        total_accuracy += accuracy(predictions, labels) / float(batch_count)
    print(np.argmax(predictions,1))
    print(np.argmax(labels,1))
    return total_accuracy

In [ ]:
def run_graph(
    graph_info,
    data,
    step_count,
    valid_count,
    test_count=0,
    report_every=50,
    verbose=True,
    accuracy_minimum=None,
    image_cache=None
):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        batch_size = graph_info["batch_size"]
        height, width, channels = data["image_size"]
        depth_labels = data["depth_labels"]
        
        inputs = np.ones(shape=(batch_size, height, width, channels), dtype=np.float32)
        labels = np.zeros(shape=(batch_size, depth_labels), dtype=np.float32)
        
        train_batcher = BatchSampler(data["train_files"], height, width, 1000)
        
        valid_files = data["valid_files"]
        valid_batcher = BatchSampler(valid_files, height, width, len(valid_files), image_cache)
        valid_accuracy = 0
        
        for step in xrange(step_count + 1):
            # Generate a minibatch.
            train_batcher.sample_batch(inputs, labels, batch_size)
            # Run the minibatch through the optimizer
            run_targets = [graph_info["optimizer"], graph_info["loss"], graph_info["predictions"]]
            feed_dict = {graph_info["train"] : inputs, graph_info["labels"] : labels}
            _, l, predictions = session.run(run_targets, feed_dict=feed_dict)
            if np.isnan(l):
                print("Error computing loss")
                return 0
            if (step % report_every == 0):
                if verbose:
                    print("Minibatch loss at step", step, ":", l)
                    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, labels))
                    print(np.argmax(predictions,1))
                    print(np.argmax(labels,1))
                valid_batcher.reset()
                valid_accuracy = batch_accuracy(session, graph_info, valid_batcher, inputs, labels, batch_size, valid_count)
                print("Validation accuracy: %.1f%%" % valid_accuracy)
                if accuracy_minimum and step > 0 and valid_accuracy < accuracy_minimum:
                    print("Early out.")
                    break

        if test_count > 0:
            test_batcher = BatchSampler(data["test_files"], height, width)
            valid_accuracy = batch_accuracy(session, graph_info, test_batcher, inputs, labels, batch_size, test_count)
            print("Test accuracy: %.1f%%" % test_results[0])
            results = results + test_results

        return valid_accuracy

In [ ]:
def create_stack(convolutions, flatten, hidden_sizes, output_size, init_mean, init_scale, l2, optimizer=None):
    stack = convevo.LayerStack(flatten=flatten, optimizer=optimizer)
    default_init = lambda: convevo.Initializer("normal", mean=init_mean, scale=init_scale)

    for operation, patch_size, stride, depth, padding, relu in convolutions:
        stack.add_layer(convevo.ImageLayer(operation, patch_size, stride, depth, "SAME", default_init(), l2_factor=l2), relu=relu)
    for hidden_size in hidden_sizes:
        stack.add_layer(convevo.HiddenLayer(hidden_size, bias=True, initializer=default_init(), l2_factor=l2), relu=True)
    if output_size is not None:
        stack.add_layer(convevo.HiddenLayer(output_size, bias=True, initializer=default_init(), l2_factor=l2), relu=False)
    
    return stack

In [ ]:
cross_data = setup_cross_validation(data_files, 9700, 400, 1000, label_count=DEPTH_LABEL_COUNT)
cross_cache = {}

In [ ]:
batch_size = 20
conv_layers = [
    ("conv_bias", 20, 2, 10, "SAME", True),
    ("conv_bias", 10, 5, 20, "SAME", True),
    ("conv_bias",  5, 2, 40, "SAME", True)
]
hidden_sizes = [400,100]
optimizer = convevo.Optimizer("Adagrad", 0.01)
prototype = create_stack(conv_layers, True, hidden_sizes, cross_data["depth_labels"], 0.0, 0.2, None, optimizer)
prototype.reseed(random)
prototype_graph = setup_graph(batch_size, cross_data["image_size"], cross_data["depth_labels"], prototype)

In [ ]:
run_graph(prototype_graph, cross_data, 100000, valid_count=20000, report_every=10000, verbose=True, image_cache=cross_cache)

In [ ]:
evo_batch_size = 40
eval_steps = 1000000
valid_steps = 20000
def eval_stack(stack, entropy):
    stack.reseed(entropy)
    
    data = setup_cross_validation(data_files, 9700, 400, 1000, label_count=DEPTH_LABEL_COUNT, seed=entropy.randint(0,24601))
    image_cache = {}
    
    try:
        evo_graph = setup_graph(evo_batch_size, data["image_size"], data["depth_labels"], stack)
    except KeyboardInterrupt:
        raise
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        print(lines[-1])
        convevo.output_error(stack, lines, "temp")
        return -10
        
    try:
        return run_graph(
            evo_graph,
            data,
            eval_steps,
            valid_count=valid_steps,
            report_every=eval_steps/4,
            verbose=False,
            image_cache=image_cache
        )
    except KeyboardInterrupt:
        raise
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        print(lines[-1])
        convevo.output_error(stack, lines, "temp")
        return -1

In [ ]:
print(convevo.serialize(prototype))
eval_stack(prototype, random.Random(42))

In [ ]:
del cross_cache
del cross_data
del conv_layers
del hidden_sizes
del prototype_graph
gc.collect()

In [ ]:
mutate_seed = random.randint(1, 100000)
print("Mutate Seed:", mutate_seed)
mutate_entropy = random.Random(mutate_seed)
eval_seed = random.randint(1, 100000)
print("Eval Seed:", eval_seed)
eval_entropy = random.Random(eval_seed)

population_size = 20
generations = 10

conv_population = convevo.init_population([prototype], population_size, mutate_entropy)
conv_darwin = darwin.Darwin(conv_population, convevo.serialize, eval_stack, convevo.breed)

for g in range(generations):
    print("Generation", g)
    results = conv_darwin.evaluate(eval_entropy)
    convevo.output_results(results, "temp")
    conv_darwin.repopulate(0.25, 4, results, mutate_entropy)